In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch

from src.config import CONFIG
from src.dataset import SBICDataset
from src.train_utils import * 
# from src.train_utils import make_tokinzer, make_model, make_dataloader, evaluate
from src.utils import print_evaluation_results
from transformers import set_seed


2023-11-17 19:42:04.674497: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 19:42:05.511487: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [70]:
config = CONFIG.hp
config.num_epochs = 200

set_seed(config.seed)

# Make the model
tokenizer = make_tokinzer(config)
model = make_model(config, tokenizer)

# Make the data
train_data = get_data("train")
val_data = get_data("validation")

# if config.train_perc != 1 or config.val_perc != 1:
#     train_size = int((config.train_perc * len(train_data)) / config.batch_size) * config.batch_size
#     val_size = int((config.val_perc * len(val_data)) / config.batch_size) * config.batch_size
#     train_idxs = np.random.choice(train_data.shape[0], train_size, replace=False)
#     val_idxs = np.random.choice(val_data.shape[0], val_size, replace=False)
    
#     train_data = train_data[train_idxs]
#     val_data = val_data[val_idxs]

train_data = train_data[9:10]
val_data = val_data[:1]

train_dataset = SBICDataset(train_data, tokenizer)
val_dataset = SBICDataset(val_data, tokenizer)

train_dataloader = make_dataloader(train_dataset, model, tokenizer, config, split="train")
val_dataloader = make_dataloader(val_dataset, model, tokenizer, config, split="validation")

# Make the loss, the optimizer and the scheduler
optimizer = make_optimizer(model, config)
scheduler = make_scheduler(
    optimizer, steps_per_epoch=len(train_dataloader), config=config
)

train(
    model,
    train_dataloader,
    val_dataloader,
    optimizer,
    scheduler,
    config,
    monitor=False,
    use_def_loss=False
)

List of all special token and its token_id:
 - ['<|endoftext|>', '<|sep|>', '<|pad|>', '<|offY|>', '<|offN|>', '<|sexY|>', '<|sexN|>', '<|intY|>', '<|intN|>', '<|grpY|>', '<|grpN|>', '<|ingrpN|>', '<|ingrpY|>']
 - [[50256], [50258], [50257], [50259], [50260], [50261], [50262], [50263], [50264], [50265], [50266], [50267], [50268]]
Model vocab resize: 50269
Model eos token: 50256
Model pad token: 50257
Model sep token: 50258
Training...


Epoch 199: 100%|█████████▉| 199/200 [00:10<00:00, 17.60epoch/s, loss=5.56e-8] 

Evaluation after the 200 steps...


Epoch 199: 100%|██████████| 200/200 [00:10<00:00, 18.35epoch/s, loss=2.38e-8]


Epoch 199:
 - loss=1.7802 def_loss=0.0000
 - avg_val_loss=17.0011, avg_val_loss_def=13.7297


In [71]:
train_data[0, 5] = [train_data[0, 5]]
train_data[0, 6] = [train_data[0, 6]]
train_data

array([['RT @iBeZo: Stupid fucking nigger LeBron. You flopping stupid jungle bunny monkey faggot.',
        1, 1, 0, 1, list(['black folks']),
        list(['black folks are stupid']), 0, 't/davidson']], dtype=object)

In [72]:
dataset = SBICDataset(train_data, tokenizer, is_training=False)
dataloader = make_dataloader(dataset, model, tokenizer, CONFIG.hp, split="validation")

res = evaluate(model, tokenizer, dataloader, CONFIG.hp)

print_evaluation_results(split="train", res=res)

Classification F1 on train set: avg=1.000
 - Offensive: 1.000
 - Intent: 1.000
 - Sex: 1.000
 - Group: 1.000
 - In-Group: 1.000
Minority RougeL-f1 on train set: 1.000
Stereotype RougeL-f1 on train set: 1.000


In [54]:
tokenizer.batch_decode([data["input_ids"] for data in train_dataloader][0], skip_special_tokens=False)

['RT @iBeZo: Stupid fucking nigger LeBron. You flopping stupid jungle bunny monkey faggot.<|sep|><|offY|><|intY|><|sexN|><|grpY|><|sep|>black folks<|sep|>black folks are stupid<|sep|><|ingrpN|><|endoftext|>']

In [73]:
tokenizer.batch_decode(res["out_tokens"], skip_special_tokens=False)

['RT @iBeZo: Stupid fucking nigger LeBron. You flopping stupid jungle bunny monkey faggot.<|sep|><|offY|><|intY|><|sexN|><|grpY|><|sep|>black folks<|sep|>black folks are stupid<|sep|><|ingrpN|><|endoftext|>']

In [3]:
tokenizer = make_tokinzer(CONFIG.hp)
model = make_model(CONFIG.hp, tokenizer)
checkpoints = "distilgpt2_full_1"
model.load_state_dict(torch.load(f"checkpoints/{checkpoints}.pt"))

List of all special token and its token_id:
 - ['<|endoftext|>', '<|sep|>', '<|pad|>', '<|offY|>', '<|offN|>', '<|sexY|>', '<|sexN|>', '<|intY|>', '<|intN|>', '<|grpY|>', '<|grpN|>', '<|ingrpN|>', '<|ingrpY|>']
 - [[50256], [50258], [50257], [50259], [50260], [50261], [50262], [50263], [50264], [50265], [50266], [50267], [50268]]
Model vocab resize: 50269
Model eos token: 50256
Model pad token: 50257
Model sep token: 50258


<All keys matched successfully>

Classification F1 on validation set: avg=0.640
 - Offensive: 0.656
 - Intent: 0.625
 - Sex: 0.757
 - Group: 0.342
 - In-Group: 0.820

Minority RougeL-f1 on validation set: 0.510

Stereotype RougeL-f1 on validation set: 0.354

In [6]:
split = "validation"
data = pd.read_pickle(CONFIG.dataset.val_data_agg).to_numpy()[:1000]
dataset = SBICDataset(data, tokenizer, is_training=False)
dataloader = make_dataloader(dataset, model, tokenizer, CONFIG.hp, split="validation")

val_res = evaluate(model, tokenizer, dataloader, CONFIG.hp)

print_evaluation_results(split=split, res=val_res)

  0%|          | 0/125 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                 

Classification F1 on validation set: avg=0.120
 - Offensive: 0.294
 - Intent: 0.053
 - Sex: 0.254
 - Group: 0.000
 - In-Group: 0.000
Minority RougeL-f1 on validation set: 0.008
Stereotype RougeL-f1 on validation set: 0.003


In [6]:
split = "test"
data = pd.read_pickle(CONFIG.dataset.test_data_agg).to_numpy()[:100]
dataset = SBICDataset(data, tokenizer, is_training=False)
dataloader = make_dataloader(dataset, model, tokenizer, CONFIG.hp, split="validation")

test_res = evaluate(model, tokenizer, dataloader, CONFIG.hp)

print_evaluation_results(split=split, res=test_res)

Classification F1 on test set: avg=0.735
 - Offensive: 0.711
 - Intent: 0.676
 - Sex: 0.886
 - Group: 0.709
 - In-Group: 0.692
Minority RougeL-f1 on test set: 0.681
Stereotype RougeL-f1 on test set: 0.421
